# 1：K-means 和PCA（主成分分析）
### **by:MLZZY**

实现K-means聚类，并利用它来压缩图像。 
先从2维数据集开始，了解K-means是如何工作的；
然后再将其应用于图像压缩。 
对主成分分析进行实验，并了解如何使用它来找到面部图像的低维表示。

## K-means 聚类

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat

实现找到数据中每个实例最接近的聚类中心的函数。

In [2]:
def find_closest_centroids(X,centroids):
    m=X.shape[0]
    k=centroids.shape[0]
    idx=np.zeros(m)
    for i in range(m):
        min_dist=1000000
        for j in range(k):
            dist=np.sum((X[i,:]-centroids[j,:])**2)
            if dist<min_dist:
                min_dist=dist
                idx[i]=j
    return idx

In [3]:
data = loadmat('/home/mw/input/andrew_ml_ex78376/ex7data2.mat')
X = data['X']
initial_centroids=np.array([[3, 3], [6, 2], [8, 5]])
idx = find_closest_centroids(X, initial_centroids)
idx

array([0., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1.

In [4]:
data2=pd.DataFrame(data['X'],columns=['X1','X2'])
data2.head()

,X1,X2
0,1.842080,4.607572
1,5.658583,4.799964
2,6.352579,3.290854
3,2.904017,4.612204
4,3.231979,4.939894


In [5]:
sb.set(context="notebook",style="white")
sb.lmplot('X1','X2',data=data2,fit_reg=False)
plt.show()

<Figure size 360x360 with 1 Axes>

实现计算簇的聚类中心的函数。 聚类中心是当前分配给簇的所有样本的平均值。

In [6]:
def compute_centroids(X,idx,k):
    m,n=X.shape
    centroids=np.zeros((k,n))
    for i in range(k):
        indices=np.where(idx==i)[0]
        centroids[i,:]=(np.sum(X[indices,:],axis=0)/len(indices))
    return centroids

In [7]:
compute_centroids(X, idx, 3)

array([[2.42830111, 3.15792418],
       [5.81350331, 2.63365645],
       [7.11938687, 3.6166844 ]])

In [8]:
def run_k_means(X,initial_centroids,max_iters):
    m,n=X.shape
    k=initial_centroids.shape[0]
    idx=np.zeros(m)
    centroids=initial_centroids
    for i in range(max_iters):
        idx=find_closest_centroids(X,centroids)
        centroids=compute_centroids(X,idx,k)
    return idx,centroids

In [9]:
idx, centroids = run_k_means(X, initial_centroids, 10)

In [10]:
cluster1 = X[np.where(idx == 0)[0],:]
cluster2 = X[np.where(idx == 1)[0],:]
cluster3 = X[np.where(idx == 2)[0],:]
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(cluster1[:,0], cluster1[:,1], s=30, color='r', label='Cluster 1')
ax.scatter(cluster2[:,0], cluster2[:,1], s=30, color='g', label='Cluster 2')
ax.scatter(cluster3[:,0], cluster3[:,1], s=30, color='b', label='Cluster 3')
ax.legend()
plt.show()

<Figure size 864x576 with 1 Axes>

以上还缺少初始化聚类中心的过程。下面实现随机选择样本并将其作为初始聚类中心的函数。

In [11]:
def init_centroids(X, k):
    m, n = X.shape
    centroids = np.zeros((k, n))
    idx = np.random.randint(0, m, k)
    for i in range(k):
        centroids[i,:] = X[idx[i],:]
    return centroids

In [12]:
init_centroids(X, 3)

array([[5.87655237, 3.21661109],
       [2.17989333, 1.30879831],
       [2.84734459, 0.26759253]])

下面将K-means用于图像压缩。
可以使用聚类来找到最具代表性的少数颜色，并使用聚类分配将原始的24位颜色映射到较低维的颜色空间。 

In [13]:
from IPython.display import Image
Image(filename='/home/mw/input/andrew_ml_ex78376/bird_small.png')

原始像素像素点数值

In [14]:
image_data = loadmat('/home/mw/input/andrew_ml_ex78376/bird_small.mat')
image_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Jun  5 04:06:24 2012',
 '__version__': '1.0',
 '__globals__': [],
 'A': array([[[219, 180, 103],
         [230, 185, 116],
         [226, 186, 110],
         ...,
         [ 14,  15,  13],
         [ 13,  15,  12],
         [ 12,  14,  12]],
 
        [[230, 193, 119],
         [224, 192, 120],
         [226, 192, 124],
         ...,
         [ 16,  16,  13],
         [ 14,  15,  10],
         [ 11,  14,   9]],
 
        [[228, 191, 123],
         [228, 191, 121],
         [220, 185, 118],
         ...,
         [ 14,  16,  13],
         [ 13,  13,  11],
         [ 11,  15,  10]],
 
        ...,
 
        [[ 15,  18,  16],
         [ 18,  21,  18],
         [ 18,  19,  16],
         ...,
         [ 81,  45,  45],
         [ 70,  43,  35],
         [ 72,  51,  43]],
 
        [[ 16,  17,  17],
         [ 17,  18,  19],
         [ 20,  19,  20],
         ...,
         [ 80,  38,  40],
         [ 68,  39,  40],
     

In [15]:
A = image_data['A']
A.shape

(128, 128, 3)

对数据进行预处理，并将其提供给K-means算法。

In [16]:
A = A / 255.
X = np.reshape(A, (A.shape[0] * A.shape[1], A.shape[2]))
X.shape

(16384, 3)

In [17]:
initial_centroids = init_centroids(X, 16)
idx, centroids = run_k_means(X, initial_centroids, 10)
idx = find_closest_centroids(X, centroids)
X_recovered = centroids[idx.astype(int),:]
X_recovered.shape

(16384, 3)

In [18]:
X_recovered = np.reshape(X_recovered, (A.shape[0], A.shape[1], A.shape[2]))
X_recovered.shape

(128, 128, 3)

In [19]:
plt.imshow(X_recovered)
plt.show()

<Figure size 432x288 with 1 Axes>

由上图可知：使用K-means对图像进行压缩后，图像的主要特征仍然存在。
下面使用scikit-learn实现K-means。

In [20]:
from skimage import io
pic = io.imread('/home/mw/input/andrew_ml_ex78376/bird_small.png') / 255.
io.imshow(pic)
plt.show()

<Figure size 432x288 with 1 Axes>

In [21]:
pic.shape

(128, 128, 3)

In [22]:
data = pic.reshape(128*128, 3)

In [23]:
data.shape

(16384, 3)

In [24]:
from sklearn.cluster import KMeans
# n_init：用不同的聚类中心初始化值运行算法的次数
# n_jobs：整形数。　指定计算所用的进程数。内部原理是同时进行n_init指定次数的计算。
#（１）若值为 -1，则用所有的CPU进行运算。若值为1，则不进行并行运算。
#（２）若值小于-1，则用到的CPU数为(n_cpus + 1 + n_jobs)。因此如果 n_jobs值为-2，则用到的CPU数为总CPU数减1。
model = KMeans(n_clusters=16, n_init=100, n_jobs=-1)

In [25]:
model.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=16, n_init=100, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [26]:
centroids = model.cluster_centers_
print(centroids.shape)
C = model.predict(data)
print(C.shape)

(16, 3)
(16384,)


In [27]:
centroids[C].shape

(16384, 3)

In [28]:
compressed_pic = centroids[C].reshape((128,128,3))

In [29]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(pic)
ax[1].imshow(compressed_pic)
plt.show()

<Figure size 432x288 with 2 Axes>

## Principal component analysis（主成分分析）

实现PCA并将其应用于二维数据集

In [30]:
data=loadmat('/home/mw/input/andrew_ml_ex78376/ex7data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Nov 14 22:41:44 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[3.38156267, 3.38911268],
        [4.52787538, 5.8541781 ],
        [2.65568187, 4.41199472],
        [2.76523467, 3.71541365],
        [2.84656011, 4.17550645],
        [3.89067196, 6.48838087],
        [3.47580524, 3.63284876],
        [5.91129845, 6.68076853],
        [3.92889397, 5.09844661],
        [4.56183537, 5.62329929],
        [4.57407171, 5.39765069],
        [4.37173356, 5.46116549],
        [4.19169388, 4.95469359],
        [5.24408518, 4.66148767],
        [2.8358402 , 3.76801716],
        [5.63526969, 6.31211438],
        [4.68632968, 5.6652411 ],
        [2.85051337, 4.62645627],
        [5.1101573 , 7.36319662],
        [5.18256377, 4.64650909],
        [5.70732809, 6.68103995],
        [3.57968458, 4.80278074],
        [5.63937773, 6.12043594],
        [4.26346851, 4.68942896],
        [2.53651693, 3.88449078],
      

In [31]:
X=data['X']
fig,ax=plt.subplots(figsize=(12,8))
ax.scatter(X[:,0],X[:,1])
plt.show()

<Figure size 864x576 with 1 Axes>

PCA算法很简单，在数据被归一化后，输出是原始数据的协方差矩阵的奇异值分解。

In [32]:
def pca(X):
    X=(X-X.mean())/X.std()
    X=np.matrix(X)
    cov=(X.T*X)/X.shape[0]
    U,S,V=np.linalg.svd(cov)
    return U,S,V

In [33]:
U, S, V = pca(X)
U, S, V

(matrix([[-0.79241747, -0.60997914],
         [-0.60997914,  0.79241747]]),
 array([1.43584536, 0.56415464]),
 matrix([[-0.79241747, -0.60997914],
         [-0.60997914,  0.79241747]]))

现在得到主成分（矩阵U），可以用这些来将原始数据投影到一个较低维的空间中。 
实现一个计算投影并且仅选择前K个分量的函数。

In [34]:
def project_data(X,U,k):
    U_reduced=U[:,:k]
    return np.dot(X,U_reduced)

In [37]:
Z = project_data(X, U, 1)
Z.shape

(50, 1)

可以通过反向转换恢复原始数据。

In [38]:
def recover_data(Z,U,k):
    U_reduced=U[:,:k]
    return np.dot(Z,U_reduced.T)

In [39]:
X_recovered = recover_data(Z, U, 1)
X_recovered

matrix([[3.76152442, 2.89550838],
        [5.67283275, 4.36677606],
        [3.80014373, 2.92523637],
        [3.53223661, 2.71900952],
        [3.80569251, 2.92950765],
        [5.57926356, 4.29474931],
        [3.93851354, 3.03174929],
        [6.94105849, 5.3430181 ],
        [4.93142811, 3.79606507],
        [5.58255993, 4.29728676],
        [5.48117436, 4.21924319],
        [5.38482148, 4.14507365],
        [5.02696267, 3.8696047 ],
        [5.54606249, 4.26919213],
        [3.60199795, 2.77270971],
        [6.58954104, 5.07243054],
        [5.681006  , 4.37306758],
        [4.02614513, 3.09920545],
        [6.76785875, 5.20969415],
        [5.50019161, 4.2338821 ],
        [6.81311151, 5.24452836],
        [4.56923815, 3.51726213],
        [6.49947125, 5.00309752],
        [4.94381398, 3.80559934],
        [3.47034372, 2.67136624],
        [4.41334883, 3.39726321],
        [5.97375815, 4.59841938],
        [6.10672889, 4.70077626],
        [4.09805306, 3.15455801],
        [4.907

In [40]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(list(X_recovered[:, 0]), list(X_recovered[:, 1]))
plt.show()

<Figure size 864x576 with 1 Axes>

将PCA应用于脸部图像，使用比原始图像少得多的数据来获取图像的“本质”。

In [41]:
faces=loadmat('/home/mw/input/andrew_ml_ex78376/ex7faces.mat')
X=faces['X']
X.shape

(5000, 1024)

In [44]:
def plot_n_image(X,n):
    pic_size=int(np.sqrt(X.shape[1]))
    grid_size=int(np.sqrt(n))
    first_n_images=X[:n,:]
    fig, ax_array = plt.subplots(nrows=grid_size, ncols=grid_size,sharey=True, sharex=True, figsize=(8, 8))
    for r in range(grid_size):
        for c in range(grid_size):
            ax_array[r,c].imshow(first_n_images[grid_size*r+c].reshape((pic_size,pic_size)).T)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))

In [45]:
plot_n_image(X,25)

<Figure size 576x576 with 25 Axes>

In [51]:
face = np.reshape(X[0,:], (32, 32))

In [52]:
plt.imshow(face)
plt.show()

<Figure size 432x288 with 1 Axes>

在人脸数据集上运行PCA，并获取前100个主要特征。

In [53]:
U, S, V = pca(X)
Z = project_data(X, U, 100)

恢复原来的结构并再次渲染。

In [55]:
X_recovered = recover_data(Z, U, 100)
face = np.reshape(X_recovered[0,:], (32, 32))
plt.imshow(face)
plt.show()

<Figure size 432x288 with 1 Axes>